## Analysis sparks multiple images

## Sparks analysis single image

In [7]:
from sparks_image import *
from sparks_analysis import *
import cv2
from scipy import stats

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imagenes_confocal/Rata/C071112/'
photo_name = 'c2ac000'
image = cv2.imread(path + photo_name + ".tif")    # Read image
list_img = image_process (image,path)
if list_img:
    list_img_col = list_img [0]
    list_img_row = list_img [1]
    out_sparks = analysis_process (list_img_col, list_img_row)

In [8]:
imCrop = list_img[2]
img_mean = (np.mean(imCrop))
img_median = (np.median(imCrop))
img_mode = stats.mode(imCrop)
img_std = (np.std(imCrop))
print(img_mean,img_median, img_mode, img_std)

25.656686626746506 25.0 ModeResult(mode=array([[[22, 22, 22],
        [15, 15, 15],
        [17, 17, 17],
        [18, 18, 18],
        [17, 17, 17],
        [17, 17, 17],
        [21, 21, 21],
        [17, 17, 17],
        [20, 20, 20],
        [18, 18, 18],
        [21, 21, 21],
        [20, 20, 20],
        [21, 21, 21],
        [21, 21, 21],
        [19, 19, 19],
        [21, 21, 21],
        [18, 18, 18],
        [21, 21, 21],
        [22, 22, 22],
        [21, 21, 21],
        [21, 21, 21],
        [21, 21, 21],
        [20, 20, 20],
        [18, 18, 18],
        [19, 19, 19],
        [21, 21, 21],
        [21, 21, 21],
        [20, 20, 20],
        [21, 21, 21],
        [22, 22, 22],
        [24, 24, 24],
        [22, 22, 22],
        [20, 20, 20],
        [24, 24, 24],
        [21, 21, 21],
        [22, 22, 22],
        [21, 21, 21],
        [20, 20, 20],
        [21, 21, 21],
        [21, 21, 21],
        [21, 21, 21],
        [19, 19, 19],
        [21, 21, 21],
        [21, 2

In [3]:
out_sparks

,tiempo_maximo,intensidad_maxima,tiempo_minimo,intensidad_minima,tiempo_valle,intensidad_valle,amplitud,TTP,TTP50,FDHM,tau,(ΔF/F0)/ΔTmax,fullDuration,fullWidth,FWHM
0,4.0,139.6875,2,99.6875,11,90.625,0.401254,2.0,0.854067,4.501617,21.026345,0.200627,9,12,3.852237
1,4.0,171.5000,1,66.5000,11,106.250,1.578947,3.0,1.535641,5.323404,14.982931,0.526316,10,11,4.263970


In [ ]:
# LinAlgError('Incompatible dimensions')

xList = np.asarray(list (range(int(out_sparks ['tiempo_maximo'] [0]), int(out_sparks ['tiempo_valle'] [0]))))
yList = np.asarray(list_img[0][0][int(out_sparks ['tiempo_maximo'] [0]) : int(out_sparks ['tiempo_valle'] [0])],dtype=np.float64)

bList = [log(max(y,1e-6)) for y in yList]
m = matrix([[1,t] for t in xList])
m2 = matrix(bList).T
(w,residuals,rank,sing_vals) = lstsq(m,m2, rcond=None)
tau = -1.0/w[1,0]
print (m)
print(m2)
# m2, n_rhs = b.shape[-2:]
# if m != m2:

from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

img_bc = interactive(f, brightness=100, contrast=100)   #display brightness and contrast
display (img_bc)

im = rotation (imCrop, 90)    # rotate image
display_image ('Image' , im)    # Display cropped image